In [34]:
import cv2
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch import optim 
import numpy as np
from facenet_pytorch import MTCNN
import mediapipe as mp
import torchvision.transforms as tt
from PIL import Image
import os

In [35]:
class Net(nn.Module):
    def __init__(self, device='cpu'):
        super().__init__()
        self.net = nn.Sequential(
          nn.Conv2d(1, 64, 3),
          nn.LeakyReLU(),
          nn.BatchNorm2d(64), 
          nn.Conv2d(64, 64, 3),
          nn.LeakyReLU(),
          nn.BatchNorm2d(64), 
          nn.MaxPool2d(2, 2), #16
          nn.Conv2d(64, 128, 3),
          nn.LeakyReLU(),          
          nn.BatchNorm2d(128), 
          nn.Conv2d(128, 128, 3),
          nn.LeakyReLU(),          
          nn.BatchNorm2d(128),                     
          nn.MaxPool2d(2, 2), #16
          nn.Conv2d(128, 256, 3),
          nn.LeakyReLU(),
          nn.BatchNorm2d(256),           
          nn.Conv2d(256, 512, 3),
          nn.Flatten(),
          nn.Linear(512, 1024),
          nn.LeakyReLU(),    
          nn.Dropout(0.2),
          nn.BatchNorm1d(1024),                 
          nn.Linear(1024, 1024),
          nn.LeakyReLU(), 
          nn.Dropout(0.2),             
          nn.BatchNorm1d(1024),                    
          nn.Linear(1024, 100), 
          nn.Softmax(dim=1)
        )
        self.mean_target_train = 0
        self.mean_target_test = 0
        self.history = None
        self.device=device

        
    def forward(self, x):
        x = self.net(x)
        return x

    
    def fit(self, loader_train, loadre_test, epochs, loss, optimazer, lr, show_log=True):
        self.train()
        self.history = []
        self.optimazer = optimazer(self.parameters(), lr)
        self.loss = loss()
        for epoch in range(epochs):
            running_loss = 0.0
            running_items = 0.0
            predict_ture = 0.0            
            for i, data in enumerate(loader_train):
                X_train, y_train = data[0].to(self.device), data[1].to(self.device)
                self.optimazer.zero_grad()
                outputs = self(X_train)
                loss_iter = self.loss(outputs, y_train)
                loss_iter.backward()
                self.optimazer.step()
                if i%200 ==0:
                    loss_test, acc_test = self.loss_fn(loader_test)
                    self.history.append([epoch+1, i+1, acc_test])
                    if show_log:
                        print(f'Epoch [{epoch + 1:2}/{epochs}]. ' \
                              f'Step [{i + 1:3}/{len(loader_train)}]. ' \
                              f'Loss_test: {loss_test:.4f}  '\
                              f'Acc_test: {acc_test:.3f}'
                              )
                running_loss, running_items = 0.0, 0.0
        print(f'Training is finished with optimazer: {optimazer}')
        return self.history

    
    def loss_fn(self, loader):
        self.eval()
        running_loss = 0.0
        running_items = 0.0
        predict_ture = 0.0        
        for i, data in enumerate(loader):
            X_data, y_data = data[0].to(self.device), data[1].to(self.device)
            outputs = self(X_data)
            loss_iter = self.loss(outputs, y_data)
            running_loss += loss_iter.item()
            _, predicted = torch.max(outputs, 1)
            predict_ture += (y_data==predicted).sum().item()
            running_items += len(y_data)
        self.train()  
        return running_loss / running_items, predict_ture/running_items

In [49]:
class FaceDetector(object):

    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'        
        # детектор лица
        self.mtcnn = MTCNN() 
        # детектор руки
        mpHands = mp.solutions.hands
        self.hands = mpHands.Hands(False)
        # загружаем модель
        self.model = torch.load('G:\\Мой диск\\Colab Notebooks\\PyTorch\\lesson10\\model_1.pth', 
                                map_location=torch.device('cpu'))
        self.model.eval()
      
        # Определяем трансформацию изображений с камеры
        self.transform = tt.Compose([tt.Grayscale(num_output_channels=1),
                         tt.ToTensor(),
                         tt.Normalize(mean=[0.485],
                                       std=[0.229])
                       ])
    
        # Параметры вывода на экран результатов
        self.font                   = cv2.FONT_HERSHEY_SIMPLEX
        self.bottomLeftCornerOfText = (10,100)
        self.fontScale              = 0.5
        self.fontColor              = (255,255,255)
        self.thickness              = 1
        self.lineType               = 2
    
    
    def face_detector(self, imgRGB):
        '''Производит детекцию лица'''
        flag = False
        boxes, probs = self.mtcnn.detect(imgRGB, landmarks=False)
        imgFace = imgRGB.copy()
        
        # если детекция успешна выдаёт положительный флаг и рисует баунтин бокс на изображении
        if boxes is not None:  
            flag = True
            boxes = boxes.astype(int)
            self._drawBoxes(imgFace, boxes, (255,0,0))
        return imgFace, flag       


    def hands_detector(self, imgRGB):
        '''Производит детекцию руки'''        
        results = self.hands.process(imgRGB)
        imgHand = imgRGB.copy()
        boxes = []    

        if results.multi_hand_landmarks:
            
            # если детекция успешна то перебираем все полученные наборы      
            for handLms in results.multi_hand_landmarks:
                x_ = []
                y_ = []
                
                # по точкам ищем границы руки, увеличиваем границы на величину padding, и 
                # получаем баунтинбокс для каждой руки
                for id, lm in enumerate(handLms.landmark):
                    x_.append(lm.x)
                    y_.append(lm.y)
                    h,w,c = imgRGB.shape
                padding = 1.2
                x_max = np.array(x_).max()*w
                x_min = np.array(x_).min()*w
                y_max = np.array(y_).max()*h
                y_min = np.array(y_).min()*h
                x_centr = (x_max+x_min)/2
                y_centr = (y_max+y_min)/2
                if x_max-x_min > y_max-y_min:
                    delta = (x_centr - x_min)*padding
                else:
                    delta = (y_centr - y_min)*padding
                    
                X_min = x_centr-delta if x_centr-delta>=0 else 0
                X_max = x_centr+delta if x_centr+delta<=w else w            
                Y_min = y_centr-delta if y_centr-delta>=0 else 0
                Y_max = y_centr+delta if y_centr+delta<=h else h 
                
                boxes.append([int(X_min), int(Y_min), int(X_max), int(Y_max)])
        
        # рисуем баунтинбоксы на изображении        
        self._drawBoxes(imgHand, boxes, (0,255,0))
        return imgHand, boxes
        
        
    def stackImages(self, scale,imgArray):
        '''Метод стекирования всех изображений в одно в виде таблицы
        scale - масштаб окна
        imgArray - список изображений dim = 1 или 2
        возвращает одно изображение в виде стека переданных ему изображений
        '''
        rows = len(imgArray)
        cols = len(imgArray[0])
        rowsAvailable = isinstance(imgArray[0], list)
        width = imgArray[0][0].shape[1]
        height = imgArray[0][0].shape[0]
        if rowsAvailable:
            for x in range ( 0, rows):
                for y in range(0, cols):
                    if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                        imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                    else:
                        imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], 
                                                                     imgArray[0][0].shape[0]), None, scale, scale)
                    if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
            imageBlank = np.zeros((height, width, 3), np.uint8)
            hor = [imageBlank]*rows
            hor_con = [imageBlank]*rows
            for x in range(0, rows):
                hor[x] = np.hstack(imgArray[x])
            ver = np.vstack(hor)
        else:
            for x in range(0, rows):
                if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                    imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
                else:
                    imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
                if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
            hor= np.hstack(imgArray)
            ver = hor
        return ver       
        
        
    def _drawBoxes(self, frame, boxes, color):
        """
        рисует баунтинбоксы на изображении
        """
        try:
            for box in boxes:
                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color, thickness=2)
        except:
            pass
        return frame

 
    def cropHand(self, imgRGB ,boxes):
        '''Вырезает из изображения первый баунтинбокс и выдаёт GRAY изображение (32,32)'''
        flag = False
        if boxes != []:
            flag = True
            imgHand = imgRGB[boxes[0][1]:boxes[0][3], boxes[0][0]:boxes[0][2]].copy()
        else:
            imgHand = np.zeros_like(imgRGB) 
        resized_image = cv2.resize(imgHand, (32, 32))
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2GRAY) 
        return resized_image, flag
    

    def run(self, cap): 
        '''Основной метод класса'''
        if (cap.isOpened()== False): 
            print("Ошибка камеры")
        while(cap.isOpened()):
            # Считываем кадр, и отображаем его по горизотали
            ret, frame = cap.read()
            frame = cv2.flip(frame,1)
            flag_face = False 
            flag_hand = False    
            
            # если кадр считался удачно
            if ret == True:
                frame_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  

                # Производим детекцию лица и рисуем баунтинбокс
                frame_RGB_face, flag_face = self.face_detector(frame_RGB)
                
                # Производим детекцию руки и рисуем баунтинбокс если нашли лицо
                if flag_face:
                    frame_RGB_hand, boxes = self.hands_detector(frame_RGB)
                else:
                    frame_RGB_hand = np.zeros_like(frame_RGB_face)
                    boxes = []
                
                # Вырезаем из изображения руку
                frame_crop_hand, flag_hand = self.cropHand(frame_RGB, boxes)
                
                # преобразуем руку в PIL формат производим трансформацию, такую же которую производили при обучении модели
                # и возвращаем обратно в формат numpy
                pil_image = Image.fromarray(frame_crop_hand)
                self.transform(pil_image)
                cv_image = np.array(pil_image)
                
                # подготавливаем и передаём изображение в модель
                cv_image = cv_image[None, None, :,:]
                class_ = np.argmax(self.model(torch.Tensor(cv_image)).detach().numpy())

                # создаём картинку для отображения результатов
                txt = ''                
                if not flag_face:
                    txt = f'No face detection'
                if flag_face & (not flag_hand):
                    txt = f'No hand detection'
                if flag_face & flag_hand:
                    txt = f'Class - {class_}'
                
                result = np.zeros((200,200,3), np.uint8)
                cv2.putText(result,txt,
                    self.bottomLeftCornerOfText,
                    self.font,
                    self.fontScale,
                    self.fontColor,
                    self.thickness,
                    self.lineType)
                # стекируем 4 изображения (изображение детектора лица, зображение детектора рук, 
                # изображение вырезанной руки, результат работы модели) 
                stak = self.stackImages(0.5, [[frame_RGB_face, frame_RGB_hand], [frame_crop_hand, result]])
                cv2.imshow('Frame', stak)                
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            else: 
                break

        cap.release()
        cv2.destroyAllWindows()

In [50]:
cap = cv2.VideoCapture(0)  
cap.set(3, 640) # Width
cap.set(4, 480) # Lenght
cap.set(10, 20) # Brightness

fcd = FaceDetector()
fcd.run(cap)